In [36]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [37]:
news = spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://140.238.153.4:8443/naver_miniproject") \
    .option("dbtable","news") \
    .option("numPartitions",5) \
    .option("user", "minip3") \
    .option("password", 'enCore1289!') \
    .load()

In [38]:
news.limit(3).show()

+---+------------+-----------+----------------------------------+-------------------+---------------------------------+--------------------+-------------------------+
| id|main_section|sub_section|                             title|           datetime|                          content|           image_src|                   byline|
+---+------------+-----------+----------------------------------+-------------------+---------------------------------+--------------------+-------------------------+
|  1|         100|        264|대통령실 민정수석 부활…시민사회...|2024-05-01 23:10:00|김주현·박찬호 등…민정수석 검토...|https://imgnews.p...|박미영 기자(mypark@new...|
|  2|         100|        264| 이르면 내주 용산 조직개편…민정...|2024-05-01 22:07:00|정무수석실도 재정비…정책·홍보 ...|https://imgnews.p...|  정아란(airan@yna.co....|
|  3|         100|        264|  윤 대통령, 다음주 공식 기자회...|2024-05-01 20:45:00| [뉴스리뷰][앵커]윤석열 대통령...|                null|  최지숙(js173@yna.co.kr)|
+---+------------+-----------+----------------------------------+-------------------+-

In [39]:
news.count()

241993

In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth,col

In [41]:
news_md = news.withColumn("month", month(col("datetime")))

In [42]:
news_md = news_md.withColumn("day", dayofmonth(col("datetime")))

In [43]:
news_md.limit(1).show()

+---+------------+-----------+----------------------------------+-------------------+---------------------------------+--------------------+-------------------------+-----+---+
| id|main_section|sub_section|                             title|           datetime|                          content|           image_src|                   byline|month|day|
+---+------------+-----------+----------------------------------+-------------------+---------------------------------+--------------------+-------------------------+-----+---+
|  1|         100|        264|대통령실 민정수석 부활…시민사회...|2024-05-01 23:10:00|김주현·박찬호 등…민정수석 검토...|https://imgnews.p...|박미영 기자(mypark@new...|    5|  1|
+---+------------+-----------+----------------------------------+-------------------+---------------------------------+--------------------+-------------------------+-----+---+



In [44]:
news_md.printSchema()

root
 |-- id: integer (nullable = true)
 |-- main_section: string (nullable = true)
 |-- sub_section: string (nullable = true)
 |-- title: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- content: string (nullable = true)
 |-- image_src: string (nullable = true)
 |-- byline: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [45]:
pip install hdfs

Note: you may need to restart the kernel to use updated packages.


In [112]:
classpath = subprocess.Popen(["/home/hadoop/hadoop/bin/hdfs", "classpath", "--glob"], stdout=subprocess.PIPE).communicate()[0]
os.environ["CLASSPATH"] = classpath.decode("utf-8")
hdfs = fs.HadoopFileSystem(host='192.168.0.251', port=8020, user='minip3')

In [116]:
hdfs.get_file_info(pa.fs.FileSelector("/.", recursive=True))  

[<FileInfo for '/test': type=FileType.Directory>,
 <FileInfo for '/user': type=FileType.Directory>,
 <FileInfo for '/user/minip3': type=FileType.Directory>,
 <FileInfo for '/user/minip3/mort': type=FileType.Directory>]

In [117]:
hdfs

In [118]:
local = fs.LocalFileSystem()

In [119]:
local

In [122]:
hdfs_base_path = 'hdfs://192.168.0.251:8020/user/minip3/mort'

In [131]:
news_temp.printSchema()

root
 |-- id: integer (nullable = true)
 |-- main_section: string (nullable = true)
 |-- sub_section: string (nullable = true)
 |-- title: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- content: string (nullable = true)
 |-- image_src: string (nullable = true)
 |-- byline: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [144]:
# sqlalchemy pymysql ->panda df로 변환->pyarrow hdfs 객체-> parquet

In [137]:
from pyspark.sql.functions import date_format

news_temp = news_temp.withColumn("datetime", date_format("datetime", "yyyy-MM-dd HH:mm:ss"))

In [140]:
news_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13885 entries, 0 to 13884
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            13885 non-null  int32 
 1   main_section  13885 non-null  object
 2   sub_section   13885 non-null  object
 3   title         13885 non-null  object
 4   datetime      13885 non-null  object
 5   content       13885 non-null  object
 6   image_src     11936 non-null  object
 7   byline        13325 non-null  object
 8   month         13885 non-null  int32 
 9   day           13885 non-null  int32 
dtypes: int32(3), object(7)
memory usage: 922.2+ KB


In [152]:
for i in range(14,16): #5월1일부터 5월 16일까지
    # print(i)
    news_temp = news_md.filter((col("month") == 5) & (col("day") == i))
    news_temp = news_temp.withColumn("datetime", date_format("datetime", "yyyy-MM-dd HH:mm:ss"))
    output_path = f"{hdfs_base_path}/news_5_{i}.parquet"
    # output_path = f"{hdfs_base_path}/month=5/day={i}/news_temp.parquet"
    hdfs = fs.HadoopFileSystem(host='192.168.0.251', port=8020, user='minip3')
    # HadoopFileSystem을 사용하여 HDFS에 저장
    with hdfs.open_output_stream(output_path) as file:
        pq.write_table(pa.Table.from_pandas(news_temp.toPandas()), file)
    # news_temp.write.parquet(output_path, mode='overwrite')
    # with local.open_output_stream(output_path) as stream:
        # stream.write(b'data')
    # hdfs_path = f"{hdfs_base_path}/month=5/day={i}"
    # hdfs_path = f"hdfs://192.168.0.251:50070/user/minip3/mort/month=5/day={i}"
    # news_temp.write.parquet(hdfs_path)
    # hdfs://192.168.0.251:8020/data/month=5/day={i}
    # output_path = f"/home/hadoop/DE31-3rd_team3/Crawler/news_temp_{i}.parquet"
    # news_temp.write.parquet(output_path, mode='overwrite')
    # os.system(f"hadoop fs -put {output_path} /user/minip3/mort/")  
    # with fs.HadoopFileSystem(host='192.168.0.251', port=8020, user='minip3') as fs1:
    #     with fs1.open_output_stream(hdfs_path) as f1:
    #         pq.write_table(news_temp, f1)
    # with fs.HadoopFileSystem(host='192.168.0.251', port=8020, user='minip3') as hdfs:
    #     with fs.open_output_stream(hdfs_path) as f:
    #             pq.write_table(news_temp,f)

In [99]:
hdfs.get_file_info(fs.FileSelector("/", recursive=True))

[<FileInfo for '/test': type=FileType.Directory>,
 <FileInfo for '/user': type=FileType.Directory>,
 <FileInfo for '/user/minip3': type=FileType.Directory>,
 <FileInfo for '/user/minip3/mort': type=FileType.Directory>]

In [100]:
hdfs.open_input_file

<bound method FileSystem.open_input_file of <pyarrow._hdfs.HadoopFileSystem object at 0x7fe238f62930>>

In [49]:
# hdfs_base_path = "hdfs://192.168.0.251:8020/home/minip3"

In [101]:
hdfs.create_dir("mort"

In [150]:
data = spark.read.parquet("hdfs://192.168.0.251:8020/user/minip3/mort/news_5_1.parquet")

In [151]:
data.show()

+---+------------+-----------+----------------------------------+-------------------+----------------------------------+--------------------+--------------------------+-----+---+
| id|main_section|sub_section|                             title|           datetime|                           content|           image_src|                    byline|month|day|
+---+------------+-----------+----------------------------------+-------------------+----------------------------------+--------------------+--------------------------+-----+---+
|  1|         100|        264|대통령실 민정수석 부활…시민사회...|2024-05-01 23:10:00| 김주현·박찬호 등…민정수석 검토...|https://imgnews.p...| 박미영 기자(mypark@new...|    5|  1|
|  2|         100|        264| 이르면 내주 용산 조직개편…민정...|2024-05-01 22:07:00| 정무수석실도 재정비…정책·홍보 ...|https://imgnews.p...|   정아란(airan@yna.co....|    5|  1|
|  3|         100|        264|  윤 대통령, 다음주 공식 기자회...|2024-05-01 20:45:00|  [뉴스리뷰][앵커]윤석열 대통령...|                null|   최지숙(js173@yna.co.kr)|    5|  1|
|  4|         